# Model Fit Comparison: LMM vs GLMCopula

In [1]:
versioninfo()

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code


In [2]:
using GLMCopula, RData, MixedModels
using LinearAlgebra, Random

┌ Info: Recompiling stale cache file /Users/huazhou/.julia/compiled/v1.2/GLMCopula/1swgR.ji for GLMCopula [e1188d9a-b146-567e-aa2b-a46eb542ab6f]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /Users/huazhou/.julia/compiled/v1.2/MixedModels/tBiYK.ji for MixedModels [ff71e718-51f3-5ec2-a782-8ffcbfa3c316]
└ @ Base loading.jl:1240


## Dyestuff Data

This is a random intercept model `Y ~ 1 + (1|G)` for a data set in R. The fitted model by the [MixedModels.jl package](https://dmbates.github.io/MixedModels.jl/stable/constructors/#Models-with-simple,-scalar-random-effects-1) yields a loglikelihood of **-163.66353**. We will see the copula model has a slightly better fit with loglikelihood **-163.35545**.

In [3]:
dat = Dict(Symbol(k)=>v for (k,v) in 
    load(joinpath(dirname(pathof(MixedModels)), "..", "test", "dat.rda")))
dyestuff = dat[:Dyestuff]

,G,Y
,Categorical…,Float64
1,A,1545.0
2,A,1440.0
3,A,1440.0
4,A,1520.0
5,A,1580.0
6,B,1540.0
7,B,1555.0
8,B,1490.0
9,B,1560.0


In [4]:
groups = unique(dyestuff[!, :G])
n, p, m = length(groups), 1, 1
gcs = Vector{GaussianCopulaVCObs{Float64}}(undef, n)
for (i, grp) in enumerate(groups)
    gidx = dyestuff[!, :G] .== grp
    ni = count(gidx)
    y = dyestuff[gidx, :Y]
    X = ones(ni, 1)
    V = [ones(ni, ni)]
    gcs[i] = GaussianCopulaVCObs(y, X, V)
end
gcm = GaussianCopulaVCModel(gcs);

In [5]:
# initialize β and τ from least square solution
init_β!(gcm)
@show gcm.β
# update σ2 and τ from β using MM algorithm
@show gcm.τ
fill!(gcm.Σ, 1)
update_Σ!(gcm)
@show gcm.Σ;

gcm.β = [1527.4999999999998]
gcm.τ = [0.0002604449267498643]
gcm.Σ = [0.8636149140056555]


In [6]:
@show loglikelihood!(gcm, true, false)
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ;

loglikelihood!(gcm, true, false) = -164.00082379403617
gcm.∇β = [0.06561148856048625]
gcm.∇τ = [-0.004475727521338513]
gcm.∇Σ = [-8.467730615246971e-6]


In [7]:
# @time GLMCopula.fit!(gcm, NLopt.NLoptSolver(algorithm=:LD_SLSQP, maxeval=4000))
@time GLMCopula.fit!(gcm, IpoptSolver(print_level=5))
@show gcm.β
@show gcm.τ
@show gcm.Σ;


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

In [8]:
@show loglikelihood!(gcm, true, false)
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ;

loglikelihood!(gcm, true, false) = -163.35545423301846
gcm.∇β = [5.498587696273205e-12]
gcm.∇τ = [1.1571319191716611e-7]
gcm.∇Σ = [1.1514215103220512e-10]


## Dyestuff2 data

`Dyestuff2` is a data set simulated from LMM. [The fitted model by MixedModels.jl package](https://github.com/dmbates/MixedModelsinJulia/blob/master/SimpleLMM.ipynb) has a log-likelihood of **-81.436518**, while copula model yields **-81.436519**.

In [9]:
dyestuff2 = dat[:Dyestuff2]

,G,Y
,Categorical…,Float64
1,A,7.298
2,A,3.846
3,A,2.434
4,A,9.566
5,A,7.99
6,B,5.22
7,B,6.556
8,B,0.608
9,B,11.788


In [10]:
groups = unique(dyestuff2[!, :G])
n, p, m = length(groups), 1, 1
gcs = Vector{GaussianCopulaVCObs{Float64}}(undef, n)
for (i, grp) in enumerate(groups)
    gidx = dyestuff2[!, :G] .== grp
    ni = count(gidx)
    y = dyestuff2[gidx, :Y]
    X = ones(ni, 1)
    V = [ones(ni, ni)]
    gcs[i] = GaussianCopulaVCObs(y, X, V)
end
gcm = GaussianCopulaVCModel(gcs);

In [11]:
# initialize β and τ from least square solution
init_β!(gcm)
@show gcm.β
@show gcm.τ
# update σ2 and τ from β using MM algorithm
fill!(gcm.Σ, 1)
update_Σ!(gcm)
@show gcm.Σ;

gcm.β = [5.6655999999999995]
gcm.τ = [0.07492826008723599]
gcm.Σ = [8.002522079028351e-7]


In [12]:
@show loglikelihood!(gcm, true, false)
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ;

loglikelihood!(gcm, true, false) = -81.43652408245762
gcm.∇β = [-7.017719738655614e-13]
gcm.∇τ = [-7.681370063394866e-5]
gcm.∇Σ = [-7.192151129476404]


In [13]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level=5))
@show gcm.β
@show gcm.τ
@show gcm.Σ;

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [14]:
@show loglikelihood!(gcm, true, false)
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ;

loglikelihood!(gcm, true, false) = -81.43651913862904
gcm.∇β = [-1.2878587085651816e-14]
gcm.∇τ = [-1.0833236917306976e-5]
gcm.∇Σ = [-7.192167502739009]


## Sleepstudy Data

Sleepstudy data has random intercept and slope within each group (individual) `Y ~ 1 + U + (1+U|G)`. The LMM model fit by [MixedModels.jl package](https://dmbates.github.io/MixedModels.jl/stable/constructors/#Models-with-vector-valued-random-effects-1) has a loglikelihood **-875.96967**.

In [15]:
sleepstudy = dat[:sleepstudy]

,Y,U,G
,Float64,Float64,Categorical…
1,249.56,0.0,308
2,258.705,1.0,308
3,250.801,2.0,308
4,321.44,3.0,308
5,356.852,4.0,308
6,414.69,5.0,308
7,382.204,6.0,308
8,290.149,7.0,308
9,430.585,8.0,308


In [16]:
groups = unique(sleepstudy[!, :G])
n, p, m = length(groups), 2, 2
gcs = Vector{GaussianCopulaLMMObs{Float64}}(undef, n)
for (i, grp) in enumerate(groups)
    gidx = sleepstudy[!, :G] .== grp
    ni = count(gidx)
    yi = Float64.(sleepstudy[gidx, :Y])
    Ui = Float64.(sleepstudy[gidx, :U])
    Xi = [ones(ni, 1) Ui]
    gcs[i] = GaussianCopulaLMMObs(yi, Xi, Xi)
end
gcm = GaussianCopulaLMMModel(gcs);

In [17]:
# initialize β and τ from least square solution
init_β!(gcm)
@show gcm.β
@show gcm.τ[1]
gcm.Σ .= diagm(ones(2))
@show gcm.Σ
@show loglikelihood!(gcm, true, false)
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ;

gcm.β = [251.40510484848488, 10.46728595959596]
gcm.τ[1] = 0.0004441684924519196
gcm.Σ = [1.0 0.0; 0.0 1.0]
loglikelihood!(gcm, true, false) = -949.1827705115786
gcm.∇β = [-0.582840426898384, -3.686754757369668]
gcm.∇τ = [37690.66749687556]
gcm.∇Σ = [0.7263046709981121 -0.22102101114869332; -0.22102101114869332 -1.86408558819219]


In [18]:
@show loglikelihood!(gcm, true, false)
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ;

loglikelihood!(gcm, true, false) = -949.1827705115786
gcm.∇β = [-0.582840426898384, -3.686754757369668]
gcm.∇τ = [37690.66749687556]
gcm.∇Σ = [0.7263046709981121 -0.22102101114869332; -0.22102101114869332 -1.86408558819219]


In [19]:
# @time GLMCopula.fit!(gcm, NLopt.NLoptSolver(algorithm=:LD_SLSQP, maxeval=4000))
@time GLMCopula.fit!(gcm, IpoptSolver(print_level=5))
@show gcm.β
@show gcm.τ
@show gcm.Σ;

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [20]:
@show loglikelihood!(gcm, true, false)
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ;

loglikelihood!(gcm, true, false) = -939.1290705973413
gcm.∇β = [1.904129631746798e-12, 4.075129123037868e-12]
gcm.∇τ = [-4.4795342546422035e-8]
gcm.∇Σ = [-0.12152695483356046 0.886030527957967; 0.886030527957967 -6.459884537297988]
